# 🌊 FlowGrad v0.3 — Complete Demo

**One-line Training Diagnostics, Feature Engineering & Model Compression for DL, ML, and RecSys**

This notebook demonstrates all FlowGrad features with real datasets (no downloads needed):

| Section | Feature | Dataset |
|---|---|---|
| 1 | sklearn GradientBoosting tracking | Breast Cancer |
| 2 | RandomForest per-tree analysis | Breast Cancer |
| 3 | XGBoost round-by-round diagnostics | California Housing |
| 4 | **Feature Engineering** — interactions, suggestions with VIF | California Housing |
| 5 | **Before/After Comparison** — prove features actually help | California Housing |
| 6 | PyTorch DL layer health tracking | Breast Cancer |
| 7 | **Model Compression** — auto pruning search & layer sensitivity ⭐ | Breast Cancer |

## 0. Installation

In [ ]:
!pip install "flowgrad[all] @ git+https://github.com/hw01931/FlowGrad.git" -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer, fetch_california_housing
from sklearn.model_selection import train_test_split

print("✅ Setup complete!")

---
## 1. 🔧 sklearn — GradientBoosting Tracking

**Dataset**: Breast Cancer (569 samples, 30 features, binary classification)

Track feature importance evolution using `warm_start` — every 10 rounds.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from flowgrad import SklearnTracker

data = load_breast_cancer()
X, y = data.data, data.target
feature_names = list(data.feature_names)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

tracker = SklearnTracker(feature_names=feature_names)
model = GradientBoostingClassifier(n_estimators=100, max_depth=3, warm_start=True, random_state=42)
tracker.track_warm_start(model, X_train, y_train, X_val, y_val, step_size=10)

print(tracker)

In [ ]:
tracker.report();

In [ ]:
fig = tracker.plot.feature_drift(top_k=8)
plt.show()

fig = tracker.plot.feature_importance_heatmap(top_k=12)
plt.show()

fig = tracker.plot.overfitting_detector()
plt.show()

---
## 2. 🌲 RandomForest — Per-Tree Analysis

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
rf_model.fit(X_train, y_train)

rf_tracker = SklearnTracker.from_forest(rf_model, feature_names=feature_names)
print(f"Trees: {rf_tracker.store.num_rounds}, Accuracy: {rf_model.score(X_val, y_val):.4f}")

fig = rf_tracker.plot.feature_drift(top_k=6)
plt.show()

---
## 3. 🌲 XGBoost — Round-by-Round Diagnostics

**Dataset**: California Housing (20,640 samples, 8 features, regression)

In [ ]:
import xgboost as xgb
from flowgrad import BoostingTracker

housing = fetch_california_housing()
X_h, y_h = housing.data, housing.target
feat_names_h = list(housing.feature_names)
X_h_train, X_h_val, y_h_train, y_h_val = train_test_split(X_h, y_h, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_h_train, label=y_h_train, feature_names=feat_names_h)
dval = xgb.DMatrix(X_h_val, label=y_h_val, feature_names=feat_names_h)

xgb_tracker = BoostingTracker()
xgb_model = xgb.train(
    {"objective": "reg:squarederror", "max_depth": 6, "learning_rate": 0.1, "subsample": 0.8},
    dtrain, num_boost_round=200,
    evals=[(dtrain, "train"), (dval, "valid")],
    callbacks=[xgb_tracker.as_xgb_callback()],
    verbose_eval=False,
)

xgb_tracker.report();

In [ ]:
fig = xgb_tracker.plot.eval_metrics()
plt.show()

fig = xgb_tracker.plot.feature_drift(top_k=8)
plt.show()

fig = xgb_tracker.plot.overfitting_detector()
plt.show()

---
## 4. 🧪 Feature Engineering — Interactions, Suggestions & VIF ⭐

FlowGrad's key differentiator: **go beyond `df.corr()`**.

- Detect non-linear feature synergies
- Suggest concrete new features (A*B, A/B, etc.)
- **NEW: VIF collinearity check** — flags suggestions that would cause multicollinearity

In [ ]:
from flowgrad import FeatureAnalyzer
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42)
rf_reg.fit(X_h_train, y_h_train)

analyzer = FeatureAnalyzer(rf_reg, X_h_train, y_h_train, feature_names=feat_names_h)
print(f"Model R² (val): {rf_reg.score(X_h_val, y_h_val):.4f}")

In [ ]:
# Feature Interactions — which pairs have synergy?
interactions = analyzer.interactions(top_k=10, method="correlation")
print("\n🔗 Top Feature Interactions:")
for i, item in enumerate(interactions, 1):
    print(f"  {i}. {item['feat_a']:>12s} × {item['feat_b']:<12s}  synergy={item['synergy_score']:+.4f}")

In [ ]:
# Feature Suggestions with VIF collinearity check
suggestions = analyzer.suggest_features(top_k=15, collinearity_check=True, vif_threshold=10.0)
print("\n💡 Suggested Features (with collinearity check):")
print(f"   {'Expression':<35s} {'Corr':>6s} {'Lift':>6s} {'VIF':>8s} {'Warning'}")
print("   " + "─" * 70)
for item in suggestions:
    if item['lift'] > 0:
        vif = item.get('vif_score', '-')
        warn = '⚠️ HIGH' if item.get('collinearity_warning', False) else '✅ OK'
        vif_str = f"{vif:>7.1f}" if isinstance(vif, float) else f"{vif:>7s}"
        print(f"   {item['expression']:<35s} {item['target_correlation']:>6.4f} {item['lift']:>+6.4f} {vif_str}  {warn}")

In [ ]:
# Redundancy + Clustering
redundant = analyzer.redundant_features(threshold=0.90)
print(f"\n🗑️ Redundant Features: {len(redundant)} pairs found")
for item in redundant[:5]:
    print(f"  {item['feat_a']} ↔ {item['feat_b']}  corr={item['correlation']:.4f}  → {item['recommendation']}")

clusters = analyzer.feature_clusters()
print(f"\n📦 Feature Clusters: {len(clusters)} clusters")
for c in clusters:
    print(f"  Cluster {c['cluster_id']} (cohesion={c['cohesion_score']:.2f}): {c['features']}")

In [ ]:
# Visualizations
fig = analyzer.plot.interaction_heatmap(top_k=8)
plt.show()

fig = analyzer.plot.suggestion_chart(top_k=8)
plt.show()

fig = analyzer.plot.cluster_map()
plt.show()

---
## 5. 📊 Before/After Comparison — Do Suggested Features Actually Help?

Take the top suggested features (filtering out high-VIF ones), add them to the dataset, retrain, and compare.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Baseline model performance
baseline_pred = rf_reg.predict(X_h_val)
baseline_mse = mean_squared_error(y_h_val, baseline_pred)
baseline_r2 = r2_score(y_h_val, baseline_pred)

print(f"📊 BASELINE (original {X_h_train.shape[1]} features):")
print(f"   MSE  = {baseline_mse:.4f}")
print(f"   R²   = {baseline_r2:.4f}")

In [ ]:
# Select top suggestions that are NOT collinear
safe_suggestions = [s for s in suggestions 
                    if s['lift'] > 0 and not s.get('collinearity_warning', False)]
top_safe = safe_suggestions[:5]

print(f"\n✅ Adding {len(top_safe)} safe features (low VIF):")
for s in top_safe:
    print(f"   + {s['expression']} (VIF={s.get('vif_score', 'N/A')})")

# Create augmented dataset
def add_features(X, suggestions, feat_names):
    X_aug = X.copy()
    new_names = list(feat_names)
    for s in suggestions:
        i = feat_names.index(s['feat_a'])
        j = feat_names.index(s['feat_b'])
        op = s['operation']
        if op == '*':
            new_col = X[:, i] * X[:, j]
        elif op == '/':
            safe = np.where(np.abs(X[:, j]) > 1e-10, X[:, j], 1e-10)
            new_col = X[:, i] / safe
        elif op == '-':
            new_col = X[:, i] - X[:, j]
        elif op == '+':
            new_col = X[:, i] + X[:, j]
        else:
            continue
        X_aug = np.column_stack([X_aug, new_col])
        new_names.append(s['expression'])
    return X_aug, new_names

X_train_aug, aug_names = add_features(X_h_train, top_safe, feat_names_h)
X_val_aug, _ = add_features(X_h_val, top_safe, feat_names_h)

print(f"\n   Features: {X_h_train.shape[1]} → {X_train_aug.shape[1]}")

In [ ]:
# Train new model with augmented features
rf_aug = RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42)
rf_aug.fit(X_train_aug, y_h_train)

aug_pred = rf_aug.predict(X_val_aug)
aug_mse = mean_squared_error(y_h_val, aug_pred)
aug_r2 = r2_score(y_h_val, aug_pred)

print(f"\n{'='*55}")
print(f"  COMPARISON: Before vs After Feature Engineering")
print(f"{'='*55}")
print(f"  {'Metric':<10s} {'Baseline':>12s} {'+ Features':>12s} {'Change':>10s}")
print(f"  {'─'*45}")
print(f"  {'MSE':<10s} {baseline_mse:>12.4f} {aug_mse:>12.4f} {aug_mse-baseline_mse:>+10.4f}")
print(f"  {'R²':<10s} {baseline_r2:>12.4f} {aug_r2:>12.4f} {aug_r2-baseline_r2:>+10.4f}")
print(f"  {'Features':<10s} {X_h_train.shape[1]:>12d} {X_train_aug.shape[1]:>12d} {X_train_aug.shape[1]-X_h_train.shape[1]:>+10d}")

if aug_r2 > baseline_r2:
    print(f"\n  ✅ Feature engineering improved R² by {(aug_r2-baseline_r2)*100:.2f}%!")
else:
    print(f"\n  ℹ️ No improvement. Consider different operations or domain-specific features.")

---
## 6. 🔬 PyTorch — DL Layer Health Tracking

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from flowgrad import FlowTracker

X_tensor = torch.FloatTensor(X_train)
y_tensor = torch.FloatTensor(y_train).unsqueeze(1)

dl_model = nn.Sequential(
    nn.Linear(30, 64), nn.ReLU(),
    nn.Linear(64, 32), nn.ReLU(),
    nn.Linear(32, 16), nn.ReLU(),
    nn.Linear(16, 1), nn.Sigmoid(),
)

criterion = nn.BCELoss()
optimizer = optim.Adam(dl_model.parameters(), lr=0.001)

tracker = FlowTracker(dl_model)

for epoch in range(50):
    optimizer.zero_grad()
    output = dl_model(X_tensor)
    loss = criterion(output, y_tensor)
    loss.backward()
    optimizer.step()
    tracker.step(loss=loss.item())

print(f"Final loss: {loss.item():.4f}")
tracker.report();

In [ ]:
fig = tracker.plot.loss()
plt.show()

fig = tracker.plot.velocity_heatmap()
plt.show()

fig = tracker.plot.health_dashboard()
plt.show()

fig = tracker.plot.full_report()
plt.show()

tracker.detach()

---
## 7. 🗜️ Model Compression — Auto Pruning Search & Layer Sensitivity ⭐

**FlowGrad v0.3's flagship feature.**

Set a performance floor → FlowGrad automatically finds the maximum compression.

```
"Keep 95% accuracy, compress as much as possible."
→ FlowGrad: "Sparsity 62% is optimal. Size reduced 62%, accuracy 97.3% retained."
```

In [ ]:
from flowgrad import CompressionTracker

# Build and train a model for compression experiments
comp_model = nn.Sequential(
    nn.Linear(30, 128), nn.ReLU(),
    nn.Linear(128, 64), nn.ReLU(),
    nn.Linear(64, 32), nn.ReLU(),
    nn.Linear(32, 1), nn.Sigmoid(),
)

criterion = nn.BCELoss()
optimizer = optim.Adam(comp_model.parameters(), lr=0.001)

for epoch in range(80):
    optimizer.zero_grad()
    output = comp_model(X_tensor)
    loss = criterion(output, y_tensor)
    loss.backward()
    optimizer.step()

print(f"Trained model | Loss: {loss.item():.4f}")

In [ ]:
# Create evaluation function
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val)

def eval_fn(model):
    model.eval()
    with torch.no_grad():
        preds = model(X_val_tensor).squeeze()
        pred_labels = (preds > 0.5).float()
        acc = (pred_labels == y_val_tensor).float().mean().item()
    model.train()
    return acc

print(f"Original accuracy: {eval_fn(comp_model):.4f}")

In [ ]:
# 🎯 Auto Compress — Goal-based search
# "Keep at least 90% of original accuracy, find max compression"
comp_tracker = CompressionTracker(comp_model, eval_fn=eval_fn)

result = comp_tracker.auto_compress(
    method="pruning",
    performance_floor=0.90,      # Keep 90%+ of original accuracy
    search_range=(0.1, 0.9),     # Search sparsity from 10% to 90%
    search_strategy="binary",    # Binary search (fast)
    precision=0.05,              # Stop when range < 5%
)

print(f"\n{'='*60}")
print(f"  Auto-Compress Result")
print(f"{'='*60}")
print(f"  Original accuracy:  {result.performance_original:.4f}")
print(f"  Compressed accuracy: {result.performance_compressed:.4f}")
print(f"  Performance retained: {result.performance_retained*100:.1f}%")
print(f"  Size: {result.size_original_mb:.2f}MB → {result.size_compressed_mb:.2f}MB ({result.size_reduction*100:.1f}% reduction)")
print(f"  Snapshots tested: {len(result.all_snapshots)}")
print(f"\n  💊 {result.recommendation}")

In [ ]:
# 📊 Compression Report
comp_tracker.report();

In [ ]:
# 📈 Tradeoff Curve — Pareto frontier of size vs performance
fig = comp_tracker.plot.tradeoff_curve()
plt.show()

# 📊 Snapshot Comparison Timeline
fig = comp_tracker.plot.compression_timeline()
plt.show()

In [ ]:
# 🔬 Layer Sensitivity — which layers can be aggressively pruned?
print("Profiling layer sensitivity (this may take a moment)...")
sensitivity = comp_tracker.layer_sensitivity(
    sparsity_levels=[0.1, 0.3, 0.5, 0.7, 0.9]
)

print("\n🔬 Layer Sensitivity Results:")
baseline = eval_fn(comp_model)
for layer_name, results in sensitivity.items():
    print(f"\n  {layer_name}:")
    for sparsity, score in results:
        drop = (baseline - score) / baseline * 100
        emoji = "🔴" if drop > 5 else "🟡" if drop > 2 else "🟢"
        print(f"    {emoji} {sparsity:.0%} pruning → score={score:.4f} (drop={drop:.1f}%)")

In [ ]:
# 🔥 Layer Sensitivity Heatmap
fig = comp_tracker.plot.layer_sensitivity_heatmap()
plt.show()

In [ ]:
# 💊 Non-uniform pruning recommendation
rec = comp_tracker.recommend_nonuniform(performance_floor=0.95)

print("\n💊 Non-Uniform Pruning Recommendation:")
print("  (Prune sensitive layers less, safe layers more)\n")
for layer, sparsity in rec.items():
    label = "AGGRESSIVE" if sparsity >= 0.7 else "MODERATE" if sparsity >= 0.3 else "CONSERVATIVE"
    emoji = "🟢" if sparsity >= 0.5 else "🟡" if sparsity >= 0.3 else "🔴"
    print(f"  {emoji} {layer:<25s} → {sparsity:.0%} pruning ({label})")

---
## Summary

| Feature | Code |
|---|---|
| DL Tracking | `FlowTracker(model)` → `.step(loss=...)` |
| XGBoost | `callbacks=[tracker.as_xgb_callback()]` |
| sklearn | `tracker.track_warm_start(model, X, y)` |
| Feature Engineering | `FeatureAnalyzer(model, X, y)` |
| VIF Check | `.suggest_features(collinearity_check=True)` |
| **Compression** | `CompressionTracker(model, eval_fn)` |
| **Auto Search** | `.auto_compress(performance_floor=0.95)` |
| **Layer Sensitivity** | `.layer_sensitivity()` |
| Report | `.report()` |
| Plots | `.plot.full_report()` |

**GitHub**: [https://github.com/hw01931/FlowGrad](https://github.com/hw01931/FlowGrad)